# Chama - Data Transformation

## 1. Problem Statement 

#### Some event data come as json files and need some transformation to be structured as tables. Convert the case.json file to 3 csv files (Refer the website for further details)

##### 1. CuratedOfferOptions.csv, 2. DynamicPriceOption.csv, 3. DynamicPriceRange.csv


## 2. Data Importation & Exploration

In [31]:
import pandas as pd
from datetime import datetime
import json
import pytz
df = pd.read_json("case.json")
df.head(10)

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 11:44:00 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
6,2021-09-05 08:03:40 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
7,2021-09-05 04:03:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
8,2021-08-18 11:44:00 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
9,2021-09-05 09:18:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


### 3. CuratedOfferOptions (file 1)

In [32]:
df_curated = df[df['EventName'] == 'CurateOffer_Result']
df_curated

,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 09:03:29 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 05:02:18 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 09:02:29 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 09:03:14 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [51]:
import pandas as pd
import json

def extract_curation_details(df):
    extracted_data = []

    for index, row in df.iterrows():
        enqueued_time_sp = pd.to_datetime(row['EnqueuedTimeUtc']).strftime('%d/%m/%Y')
        
        payloads = row['Payload']  # Access payload
        if isinstance(payloads, str):  
            try:
                payloads = json.loads(payloads)  # Convert string to JSON
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON at row {index}: {e}")
                continue  # Skip invalid rows
        
        # Ensure payloads is a list
        if isinstance(payloads, dict):  
            payloads = [payloads]  # Convert single object to list
        
        for payload in payloads:
            if not isinstance(payload, dict):
                print(f"Skipping malformed payload at row {index}: {payload}")
                continue  # Skip malformed payloads
            
            curation_provider = payload.get("curationProvider", "")
            offer_id = payload.get("offerId", "")
            dealer_id = payload.get("dealerId", "")

            for option in payload.get("options", []):
                extracted_data.append({
                    "CurationProvider": curation_provider,
                    "OfferId": offer_id,
                    "DealerId": dealer_id,
                    "UniqueOptionId": option.get("uniqueOptionId", ""),
                    "OptionId": option.get("optionId", ""),
                    "IsMobileDealer": option.get("isMobileDealer", False),
                    "IsOpen": option.get("isOpen", False),
                    "Eta": option.get("eta", ""),
                    "ChamaScore": option.get("chamaScore", 0),
                    "ProductBrand": option.get("productBrand", ""),
                    "IsWinner": option.get("isWinner", False),
                    "MinimumPrice": option.get("minimumPrice", 0),
                    "MaximumPrice": option.get("maximumPrice", 0),
                    "DynamicPrice": option.get("dynamicPrice", 0),
                    "FinalPrice": option.get("finalPrice", 0),
                    "DefeatPrimaryReason": option.get("defeatPrimaryReason", ""),
                    "DefeatReasons": option.get("defeatReasons", []),
                    "EnqueuedTimeSP": enqueued_time_sp
                })

    return pd.DataFrame(extracted_data)

df_extracted = extract_curation_details(df_curated)
df_extracted


,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,6517,b0e296a9-0590-f0e0-8211-243a2ededb12,6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2,True,True,1:00,8.0,ULTRAGAZ,True,90.00,180.00,91.90,91.90,,[],25/08/2021
1,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,6517,d6562c24-0b37-5fb4-8275-65b7b8b47b87,6517 || 6517,False,False,0:01,8.0,ULTRAGAZ,False,90.00,180.00,91.90,91.90,Closed,"[Closed, HasDriverInOffer]",25/08/2021
2,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,9047,8d0f9262-f543-d0c8-a869-33985ae3ecda,9047 || 9047 || ULTRAGAZ,False,False,1:00,9.0,ULTRAGAZ,False,99.00,198.00,99.95,99.95,Closed,"[Closed, HigherPrice, HasDriverInOffer]",25/08/2021
3,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,9047,3cd346f4-d297-7568-2e50-d43a8e2fd0a9,9047 || 9047 || CONSIGAZ,False,False,1:00,9.0,CONSIGAZ,False,89.99,179.98,91.89,91.89,Closed,"[Closed, HigherPrice, HigherETA]",25/08/2021
4,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,9047,577e4bbd-f49d-ac23-56a6-e70072a05229,9047 || 9047 || LIQUIGAS,False,False,1:00,9.0,LIQUIGAS,False,92.00,184.00,93.90,93.90,Closed,"[Closed, HigherPrice, HigherETA]",25/08/2021
5,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,9047,f9b876ab-2590-952f-d69d-5b352ec251f3,9047 || 9047 || NACIONALGAS,False,False,1:00,9.0,NACIONALGAS,True,88.90,177.80,90.80,90.80,,[],25/08/2021
6,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,8330,151e59ac-761a-96f5-d2b9-882037a9fd28,8330 || 8330 || CONSIGAZ,False,True,0:01,8.0,CONSIGAZ,True,80.00,160.00,87.35,87.35,,[],25/08/2021
7,ByPrice,149f0e53-ff85-425f-a01a-8710f06704ea,8835,b7a7b6d1-4dae-7392-5aaf-f3369c29db1d,8835 || 8835 || LIQUIGAS,False,False,0:01,8.0,LIQUIGAS,True,60.00,120.00,87.35,87.35,,[],25/08/2021
8,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,6517,b0e296a9-0590-f0e0-8211-243a2ededb12,6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2,True,True,00:12:54.9215999,8.0,ULTRAGAZ,True,90.00,180.00,91.90,91.90,,[],25/08/2021
9,ByPrice,c99aa9a7-ac50-4a10-be0f-ac9f5ac0f45d,6517,d6562c24-0b37-5fb4-8275-65b7b8b47b87,6517 || 6517,False,True,0:01,8.0,ULTRAGAZ,False,90.00,180.00,91.90,91.90,HasDriverInOffer,[HasDriverInOffer],25/08/2021


In [52]:
csv_filename = "CuratedOffers.csv"
df_extracted.to_csv(csv_filename, index=False)

### 4. DynamicPriceOption (file 2)

In [54]:
df_dynamicPrice = df[df['EventName'] == 'DynamicPrice_Result']
df_dynamicPrice.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 11:44:00 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [60]:
import json
import pandas as pd

def extract_dynamic_price_details(df):
    extracted_data = []

    for _, row in df.iterrows():
        # Convert date format to DD/MM/YY
        enqueued_time_sp = pd.to_datetime(row['EnqueuedTimeUtc']).strftime('%d/%m/%y')
        
        # Ensure Payload is properly parsed
        payload_row = row['Payload']
        if isinstance(payload_row, str):  # Convert if it's a string
            payload_row = json.loads(payload_row)
        
        if isinstance(payload_row, dict) and payload_row.get("provider") == "ApplyDynamicPricePerOption":
            provider = f"\"{payload_row['provider']}\""
            offer_id = f"\"{payload_row['offerId']}\""

            for unique_option in payload_row.get("algorithmOutput", []):  # Handle missing keys safely
                price_option = {
                    "Provider": provider,
                    "OfferId": offer_id,
                    "UniqueOptionId": f"\"{unique_option.get('uniqueOptionId', '')}\"",
                    "BestPrice": unique_option.get("bestPrice", 0),  # Default to 0 if missing
                    "EnqueuedTimeSP": enqueued_time_sp
                }
                extracted_data.append(price_option)

    return pd.DataFrame(extracted_data)

# Apply function
df_dynamicPrice_extracted = extract_dynamic_price_details(df_dynamicPrice)
df_dynamicPrice_extracted


,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/21
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/21
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/21
5,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""",93.00,18/08/21
6,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",93.00,18/08/21
7,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""f9b876ab-2590-952f-d69d-5b352ec251f3""",91.35,18/08/21
8,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
9,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21


In [61]:
csv_filename = "DynamicPriceOptions.csv"
df_extracted.to_csv(csv_filename, index=False)

### 5. DynamicPriceRange (file 3)

In [63]:
import json
import pandas as pd
from datetime import datetime, timedelta

def extract_dynamic_price_range(df):
    extracted_data = []

    for _, row in df.iterrows():
        # Convert timestamp to Brazilian timezone (UTC-3) and format as DD/MM/YYYY
        enqueued_time_sp = (pd.to_datetime(row['EnqueuedTimeUtc']) - timedelta(hours=3)).strftime('%d/%m/%Y')
        
        # Parse payload
        payload_row = row['Payload']
        if isinstance(payload_row, str):  # Convert if it's a string
            payload_row = json.loads(payload_row)
        
        if isinstance(payload_row, dict) and payload_row.get("provider") == "ApplyDynamicPriceRange":
            provider = f"\"{payload_row['provider']}\""
            offer_id = f"\"{payload_row['offerId']}\""
            
            # Extract relevant fields
            min_global = payload_row.get("minGlobal", 0)
            min_recommended = payload_row.get("minRecommended", 0)
            max_recommended = payload_row.get("maxRecommended", 0)
            diff_min_recommend_min_theory = payload_row.get("differenceMinRecommendMinTheory", 0)
            
            price_range_data = {
                "Provider": provider,
                "OfferId": offer_id,
                "MinGlobal": min_global,
                "MinRecommended": min_recommended,
                "MaxRecommended": max_recommended,
                "DifferenceMinRecommendMinTheory": diff_min_recommend_min_theory,
                "EnqueuedTimeSP": enqueued_time_sp
            }
            
            extracted_data.append(price_range_data)

    return pd.DataFrame(extracted_data)

# Apply function
df_dynamicPriceRange_extracted = extract_dynamic_price_range(df_dynamicPrice)
df_dynamicPriceRange_extracted


,Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
0,"""ApplyDynamicPriceRange""","""a6611d55-9624-4381-8cdd-323ee3689241""",0,0,0,0,05/09/2021
1,"""ApplyDynamicPriceRange""","""b8c636fa-8241-47dc-ac40-bdf438a04d9c""",0,0,0,0,05/09/2021
2,"""ApplyDynamicPriceRange""","""3d32f7fb-396d-4d3f-b673-dea1f7dc41b7""",0,0,0,0,05/09/2021
3,"""ApplyDynamicPriceRange""","""329194f3-95a4-45ef-b3d0-2796f74ce2a0""",0,0,0,0,05/09/2021
4,"""ApplyDynamicPriceRange""","""fdcfde5c-113d-4a59-9ae0-8bc31e2943d8""",0,0,0,0,05/09/2021
5,"""ApplyDynamicPriceRange""","""27bbc4fa-2388-4780-b66c-92a51397d191""",0,0,0,0,05/09/2021
6,"""ApplyDynamicPriceRange""","""baffc30b-7642-45fe-a2ce-da31a71732ae""",0,0,0,0,05/09/2021
7,"""ApplyDynamicPriceRange""","""b5982abd-f602-47ac-b45a-bb43bf993d46""",0,0,0,0,05/09/2021
8,"""ApplyDynamicPriceRange""","""f6643886-4a0f-45ae-ae32-ee95c72ee94a""",0,0,0,0,05/09/2021
9,"""ApplyDynamicPriceRange""","""16a2d492-b1c3-40ec-970f-b8704d8db96f""",0,0,0,0,05/09/2021


In [64]:
csv_filename = "DynamicPriceRange.csv"
df_extracted.to_csv(csv_filename, index=False)